# Modelos de prediccion
Vamos a probar distintos enfoques:
Enfoques de serie de tiempo:
* Suavizado exponencial #investigar

Enfoques Machine Learning:
* XGboost: Emiii
* LIGHT_GBM: Rugo
* Catboost: Vano

In [1]:
from DataLoader import importar_ventas,importar_cluster_stores,leer_csv,agregar_fourier,preparar_test
from DataLoader import crear_archivo_kaggle
import xgboost as xgb
import pandas as pd

In [2]:
ventas=importar_ventas()

In [3]:
test=preparar_test("ids_test.csv")
#test

In [4]:
agregar_fourier(ventas,2)
agregar_fourier(test,2)


In [5]:
#busquemos otra fecha para probar esto objetivo 01-01-2023 a 07-01-2023

# ventas_2023=ventas[ventas["date"]<"02-01-2023"]
# test_2023=ventas_2023.groupby(["subgroup_cod","store_cod"]).tail(7)
# train_2023=ventas_2023.drop(test_2023.index)

In [6]:

FEATURES=[
    #'mean_price', 
    #'cluster', 
    #'store_cod', 
    #'subgroup_cod',
    #'demand_lag_1', #'demand_lag_2', 'demand_lag_3', 
    #'demand_lag_4','demand_lag_5', 'demand_lag_6', 'demand_lag_7', 'day', 'month', 
    'year',
   'anual_sin_1', 
    'anual_cos_1',
   # 'mensual_sin_1', 
    #'mensual_cos_1',
   #'anual_sin_2', 
    #'anual_cos_2',
    'mensual_sin_2', 
    #'mensual_cos_2',
    "std_price",
    "max_price", 
   "min_price"
] #recordar que ya probamos con fourier 3 y 4 y fue un fracaso

TARGET=['demand']

test_interno=ventas.groupby(["subgroup_cod","store_cod"]).tail(7)
train=ventas
train_interno=ventas.drop(test_interno.index)

X_train=train[FEATURES]
y_train=train[TARGET]
X_test_real=test[FEATURES]

X_train_interno=train_interno[FEATURES]
y_train_interno=train_interno[TARGET]

X_test_interno=test_interno[FEATURES]
y_test_interno=test_interno[TARGET]

# X_test_2023=test_2023[FEATURES]
# y_test_2023=test_2023[TARGET]

# X_train_2023=train_2023[FEATURES]
# y_train_2023=train_2023[TARGET]

## Modelo interno

In [7]:
from sklearn.metrics import r2_score

reg = xgb.XGBRegressor(
    n_estimators=50,
    eval_metric=r2_score,
    eta=0.3,
    max_depth=8,
   #colsample_bytree=0.9, #reducir el numero de columnas aumenta la importancia de std_price
    #early_stopping_rounds=600 
)

reg.fit(
    X_train, y_train
   #,eval_set=[(X_train_interno, y_train_interno)]   
   ,verbose=50                       
)
#sin anual_sin_2 450 estimadores =0.11283 y con da lo mismo

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.3,
             eval_metric=<function r2_score at 0x000001F650B4BE20>,
             feature_types=None, feature_weights=None, gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None, ...)

In [8]:
pd.DataFrame(reg.feature_importances_,columns=["importance"],index=reg.feature_names_in_).sort_values(by="importance",ascending=False)

,importance
std_price,0.617150
year,0.114090
anual_sin_1,0.098188
mensual_sin_2,0.069702
min_price,0.037531
anual_cos_1,0.036519
max_price,0.026819


## Modelo kraggel

In [9]:
prediccion=reg.predict(X_test_real)
kaggle_correcto = pd.DataFrame({
    "store_subgroup_date_id": test["store_subgroup_date_id"],
    "prediccion": prediccion.ravel()
})

kaggle=crear_archivo_kaggle(kaggle_correcto)

In [10]:
kaggle.columns
(kaggle["STORE_SUBGROUP_DATE_ID"]==kaggle_correcto["store_subgroup_date_id"]).sum()

np.int64(14)

In [11]:
test=leer_csv("ids_test")
test=test.merge(kaggle_correcto, on="store_subgroup_date_id", how="left")
(kaggle["STORE_SUBGROUP_DATE_ID"]==test["store_subgroup_date_id"]).sum() # ahi anda bien

np.int64(77700)

In [12]:
kaggle.describe()

,PREDICCION
count,77700.000000
mean,4.395554
std,1.826181
min,3.004492
25%,3.300464
50%,3.501493
75%,4.501063
max,15.500337
